In [26]:
import re
import json
from json import JSONEncoder
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

In [2]:
d = ""
with open("data/tempDataIDS.json") as file:
    d = json.loads(file.read())

In [27]:
options = Options()
options.headless = True
driver = webdriver.Firefox(firefox_options=options, executable_path='C:\Program Files\Mozilla Firefox\geckodriver.exe')
print("Initializing Headless Driver")

Initializing Headless Driver


In [4]:
print(d)

 'Ring1': {'itemID': 21601}, 'Ring2': {'itemID': 21205}, 'Trinket1': {'itemID': 19406}, 'Trinket2': {'itemID': 13966}, 'MainHand': {'itemID': 21268}, 'OffHand': {'itemID': 13385}, 'Ranged': {'itemID': 23198}}, 'FeralTank-P6': {'Head': {'itemID': 21693}, 'Neck': {'itemID': 22732}, 'Shoulder': {'itemID': 19389}, 'Cloak': {'itemID': 22938}, 'Chest': {'itemID': 23226}, 'Wrist': {'itemID': 22663}, 'Gloves': {'itemID': 21672}, 'Waist': {'itemID': 21675}, 'Legs': {'itemID': 22740}, 'Boots': {'itemID': 19381}, 'Ring1': {'itemID': 23018}, 'Ring2': {'itemID': 21601}, 'Trinket1': {'itemID': 22954}, 'Trinket2': {'itemID': 19406}, 'MainHand': {'itemID': 21268}, 'OffHand': {'itemID': 13385}, 'Ranged': {'itemID': 23198}}, 'FeralDps-P5': {'Head': {'itemID': 8345}, 'Neck': {'itemID': 19377}, 'Shoulder': {'itemID': 21665}, 'Cloak': {'itemID': 21710}, 'Chest': {'itemID': 21680}, 'Wrist': {'itemID': 21602}, 'Gloves': {'itemID': 21672}, 'Waist': {'itemID': 21586}, 'Legs': {'itemID': 16450}, 'Boots': {'item

In [5]:
class Item:
    def __init__(self, id, npcID, npcName, kill, quest, questID, recipe, recipeID, dropChance, zone):
        self.id = id
        self.Obtain = {}
        self.Obtain["NpcID"] = npcID
        self.Obtain["NpcName"] = npcName
        self.Obtain["Kill"] = kill
        self.Obtain["Quest"] = quest
        self.Obtain["QuestID"] = questID
        self.Obtain["Recipe"] = recipe
        self.Obtain["RecipeID"] = recipeID
        self.Obtain["DropChance"] = dropChance
        self.Obtain["Zone"] = zone
    def toJson(self):
        return json.dumps(self, default=lambda o: o.__dict__)

In [28]:
def ItemIsObtainType(val, ty):
    return val["Obtain"]["Type"] == ty

def GetItemFromOldDataSlot(itemID):
    url = "https://classic.wowhead.com/item=" + str(itemID)
    print(url)
    driver.get(url)
    page = driver.page_source

    soup = BeautifulSoup(page)

    rewardTab = soup.find('div',id='jkbfksdbl4')
    selectedText = rewardTab.find('a', class_='selected')['href']

    rows = soup.find(id='lkljbjkb574')
    firstRow = rows.find('tr', class_='listview-row')
    o = ""
    if (selectedText == "#dropped-by") :
        npcATag = firstRow.find('a', class_='listview-cleartext')
        npcLink = npcATag['href']
        npcID = int(re.search("(?<=\/npc=)(.*)(?=\/)", npcLink).group(0))
        npcLocation = ""
        npcLocationTag = firstRow.find('a', class_='q1')
        if (npcLocationTag is not None):
            npcLocation = npcLocationTag.text
        dropChance = firstRow.find_all('td')[6].text
        npcName = npcATag.text
        o = Item(itemID, npcID, npcName, True, False, 0, False, 0, dropChance, npcLocation)
    elif (selectedText == "#created-by-spell"):
        recipeATag = firstRow.find('a', class_='listview-cleartext')
        recipeLink = recipeATag['href']
        recipeID = int(re.search("(?<=\/spell=)(.*)(?=\/)", recipeLink).group(0))
        o = Item(itemID, 0, "", False, False, 0, True, recipeID, 0, "")
    elif (selectedText == "#reward-from-q"):
        questATag = firstRow.find('a', class_="listview-cleartext")
        questLink = questATag['href']
        questID = int(re.search("(?<=\/quest=)(.*)(?=\/)", questLink).group(0))
        questLocation = ""
        questLocationTag = firstRow.find_all('td')[8].find('a')
        if (questLocationTag is not None):
            questLocation = questLocationTag.text
        dropChance = firstRow.find_all('td')[6].text
        o = Item(itemID, 0, "", False, True, questID, False, 0, 0, questLocation)
    else:
        o = Item(itemID, 0, "", False, False, 0, False, 0, 0, "")

    
    

    print("done with " + str(itemID))
    return o.toJson()

In [29]:
nd = {}
for k, v in d.items():
    print(k)
    nd[k] = {}
    for l, b in v.items():
        print(l)
        nd[k][l] = {}
        nd[k][l]["Head"] = GetItemFromOldDataSlot(d[k][l]["Head"]["itemID"])
        nd[k][l]["Neck"] = GetItemFromOldDataSlot(d[k][l]["Neck"]["itemID"])
        nd[k][l]["Shoulder"] = GetItemFromOldDataSlot(d[k][l]["Shoulder"]["itemID"])
        nd[k][l]["Back"] = GetItemFromOldDataSlot(d[k][l]["Cloak"]["itemID"])
        nd[k][l]["Chest"] = GetItemFromOldDataSlot(d[k][l]["Chest"]["itemID"])
        nd[k][l]["Wrists"] = GetItemFromOldDataSlot(d[k][l]["Wrist"]["itemID"])
        nd[k][l]["Hands"] = GetItemFromOldDataSlot(d[k][l]["Gloves"]["itemID"])
        nd[k][l]["Waist"] = GetItemFromOldDataSlot(d[k][l]["Waist"]["itemID"])
        nd[k][l]["Legs"] = GetItemFromOldDataSlot(d[k][l]["Legs"]["itemID"])
        nd[k][l]["Feet"] = GetItemFromOldDataSlot(d[k][l]["Boots"]["itemID"])
        nd[k][l]["Finger"] = GetItemFromOldDataSlot(d[k][l]["Ring1"]["itemID"])
        nd[k][l]["RFinger"] = GetItemFromOldDataSlot(d[k][l]["Ring2"]["itemID"])
        nd[k][l]["Trinket"] = GetItemFromOldDataSlot(d[k][l]["Trinket1"]["itemID"])
        nd[k][l]["RTrinket"] = GetItemFromOldDataSlot(d[k][l]["Trinket2"]["itemID"])
        nd[k][l]["MainHand"] = GetItemFromOldDataSlot(d[k][l]["MainHand"]["itemID"])
        nd[k][l]["SecondaryHand"] = GetItemFromOldDataSlot(d[k][l]["OffHand"]["itemID"])
        nd[k][l]["Relic"] = GetItemFromOldDataSlot(d[k][l]["Ranged"]["itemID"])
print("---Done with scrape---")

Druid
FeralTank-P5
https://classic.wowhead.com/item=21693
done with 21693
https://classic.wowhead.com/item=22732
done with 22732
https://classic.wowhead.com/item=19389
done with 19389
https://classic.wowhead.com/item=19386
done with 19386
https://classic.wowhead.com/item=19405
done with 19405
https://classic.wowhead.com/item=21602
done with 21602
https://classic.wowhead.com/item=21672
done with 21672
https://classic.wowhead.com/item=21675
done with 21675
https://classic.wowhead.com/item=22740
done with 22740
https://classic.wowhead.com/item=19381
done with 19381
https://classic.wowhead.com/item=21601
done with 21601
https://classic.wowhead.com/item=21205
done with 21205
https://classic.wowhead.com/item=19406
done with 19406
https://classic.wowhead.com/item=13966
done with 13966
https://classic.wowhead.com/item=21268
done with 21268
https://classic.wowhead.com/item=13385
done with 13385
https://classic.wowhead.com/item=23198
done with 23198
FeralTank-P6
https://classic.wowhead.com/item=

KeyboardInterrupt: 

In [22]:
nd = json.dumps(nd)
nd = nd.replace('"{', '{')
nd = nd.replace('}"', '}')
nd = nd.replace('\\', '')
print(nd)

13", "Zone": "Naxxramas"}}}}, "Warrior": {"Fury-P5": {"Head": {"id": 12640, "Obtain": {"NpcID": 0, "NpcName": "", "Kill": false, "Quest": false, "QuestID": 0, "Recipe": true, "RecipeID": "16729", "DropChance": 0, "Zone": ""}}, "Neck": {"id": 21664, "Obtain": {"NpcID": "15510", "NpcName": "Fankriss the Unyielding", "Kill": true, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": "23", "Zone": "Ahn'Qiraj"}}, "Shoulder": {"id": 21330, "Obtain": {"NpcID": 0, "NpcName": "", "Kill": false, "Quest": true, "QuestID": "8544", "Recipe": false, "RecipeID": 0, "DropChance": 0, "Zone": "Ahn'Qiraj"}}, "Back": {"id": 21701, "Obtain": {"NpcID": "15263", "NpcName": "The Prophet Skeram", "Kill": true, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": "23", "Zone": "Ahn'Qiraj"}}, "Chest": {"id": 21331, "Obtain": {"NpcID": 0, "NpcName": "", "Kill": false, "Quest": true, "QuestID": "8562", "Recipe": false, "RecipeID": 0, "DropChance": 0, "Zone": "Ahn'Qiraj

In [23]:
t = open("dataFile.json", "w")
t.write(nd)
t.close()